### Xử lí data

#### 1. Xử lí data từ json

In [1]:
#Import thư viện cần thiết
import json
import pandas as pd
from itertools import zip_longest

In [2]:
rows = []

# Đọc file JSON
with open('../data/json/JSON2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lọc id trong khoảng 4653-9255
data = [item for item in data if 4653 <= item.get('id', -1) <= 9255]

# Xử lý từng item
for item in data:
    base_id = item.get('id')
    zh_text = item.get('zh', '')
    vi_text = item.get('vi', '')

    # Tách dòng
    zh_lines = zh_text.split('\n')
    vi_lines = vi_text.split('\n')

    line_number = 1  # counter cho mỗi id

    for zh, vi in zip_longest(zh_lines, vi_lines, fillvalue=''):
        new_id = f"{base_id}_{line_number}"
        line_number += 1

        rows.append({
            'src_id': new_id,
            'src_lang': zh,
            'tgt_lang': vi
        })

# Xuất thành DataFrame
df = pd.DataFrame(rows)


In [3]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    4603 non-null   object
 1   src_lang  4603 non-null   object
 2   tgt_lang  4603 non-null   object
dtypes: object(3)
memory usage: 108.0+ KB


,src_id,src_lang,tgt_lang
0,4653_1,我认为这将是主要的事情。我认为，同意某人所说，应该选择HSV-1血清阳性且携带APOE4等位...,"Tôi nghĩ rằng đó sẽ là điều chính. Tôi nghĩ, đ..."
1,4654_1,我非常想说这个。我相信这是从许多问题中得出的结论。史蒂文·雅各布森：但是马克，我可以提个建议吗？,Tôi rất muốn nói điều này. Tôi chắc chắn điều ...
2,4655_1,对我来说，这不是一个试验设计。关键不在于它是否有效，而是如果它无效，你会怎么做？,"Đối với tôi, đó không phải là một thiết kế thử..."
3,4656_1,你是否会抛弃这个假设？我会说，这不是你想要做的。如果他进行了一项大型试验，但没有成功，你会说...,Bạn có loại bỏ giả thuyết này không? Tôi sẽ nó...
4,4657_1,我认为那是不正确的。如果它有效，当然，每个人都会很高兴。麦克·马基维奇：现在，你有。史蒂文·...,Tôi không nghĩ điều đó là chính xác. Nếu nó có...
5,4658_1,它们不会恶化，也不会发展成痴呆症或阿尔茨海默病，当然你找到了。但如果你没有找到，这就是我们遇...,"Chúng không trở nên tồi tệ hơn, chúng không ti..."
6,4659_1,无论哪个假设成立，零假设确实有效，我们会说，是的，你确实发现了一些东西，或者你实际上没有发现...,"Bất kể giả thuyết nào đúng, giả thuyết null th..."
7,4660_1,其实金钱问题不在于找不到东西。露丝·伊扎奇：这很重要，因为我不知道该说什么了。,Vấn đề không phải là tìm kiếm một thứ gì đó th...
8,4661_1,史蒂文·雅各布森：是的，没错。弗雷德·沃林斯基：史蒂夫，我能说点什么吗？史蒂文·雅各布森：可...,Steven Jacobson: Đúng vậy. Fred Volinsky: Tôi ...
9,4662_1,这是弗雷德·沃林斯基，我写了一篇文章，我和帕斯卡尔·戈德施密特合作撰写，他阅读了这篇文章，我...,"Đây là Fred Volinsky, và tôi đã viết một bài b..."


In [4]:
# Xóa các dòng chứa kí tự "-,-", ",-" hoặc "-,"
remove_mask = (
    (df["src_lang"] == "-") |
    (df["tgt_lang"] == "-") |
    (df["src_lang"] + "," + df["tgt_lang"]).isin(["-,-", ",-", "-,"])
)

df_clean = df[~remove_mask].copy()

new_ids = []
current_prefix = None
counter = 1

for old_id in df_clean["src_id"]:
    prefix, num = old_id.split("_")

    if current_prefix != prefix:
        current_prefix = prefix
        counter = 1

    new_ids.append(f"{prefix}_{counter}")
    counter += 1

df_clean.loc[:, "src_id"] = new_ids
df_clean = df_clean.reset_index(drop=True)

# Ghi đè vào file gốc
df_clean.to_csv("../data/preprocessing_data/json2.csv", index=False, encoding="utf-8-sig")

df_clean.info()
df_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    4603 non-null   object
 1   src_lang  4603 non-null   object
 2   tgt_lang  4603 non-null   object
dtypes: object(3)
memory usage: 108.0+ KB


,src_id,src_lang,tgt_lang
0,4653_1,我认为这将是主要的事情。我认为，同意某人所说，应该选择HSV-1血清阳性且携带APOE4等位...,"Tôi nghĩ rằng đó sẽ là điều chính. Tôi nghĩ, đ..."
1,4654_1,我非常想说这个。我相信这是从许多问题中得出的结论。史蒂文·雅各布森：但是马克，我可以提个建议吗？,Tôi rất muốn nói điều này. Tôi chắc chắn điều ...
2,4655_1,对我来说，这不是一个试验设计。关键不在于它是否有效，而是如果它无效，你会怎么做？,"Đối với tôi, đó không phải là một thiết kế thử..."
3,4656_1,你是否会抛弃这个假设？我会说，这不是你想要做的。如果他进行了一项大型试验，但没有成功，你会说...,Bạn có loại bỏ giả thuyết này không? Tôi sẽ nó...
4,4657_1,我认为那是不正确的。如果它有效，当然，每个人都会很高兴。麦克·马基维奇：现在，你有。史蒂文·...,Tôi không nghĩ điều đó là chính xác. Nếu nó có...
5,4658_1,它们不会恶化，也不会发展成痴呆症或阿尔茨海默病，当然你找到了。但如果你没有找到，这就是我们遇...,"Chúng không trở nên tồi tệ hơn, chúng không ti..."
6,4659_1,无论哪个假设成立，零假设确实有效，我们会说，是的，你确实发现了一些东西，或者你实际上没有发现...,"Bất kể giả thuyết nào đúng, giả thuyết null th..."
7,4660_1,其实金钱问题不在于找不到东西。露丝·伊扎奇：这很重要，因为我不知道该说什么了。,Vấn đề không phải là tìm kiếm một thứ gì đó th...
8,4661_1,史蒂文·雅各布森：是的，没错。弗雷德·沃林斯基：史蒂夫，我能说点什么吗？史蒂文·雅各布森：可...,Steven Jacobson: Đúng vậy. Fred Volinsky: Tôi ...
9,4662_1,这是弗雷德·沃林斯基，我写了一篇文章，我和帕斯卡尔·戈德施密特合作撰写，他阅读了这篇文章，我...,"Đây là Fred Volinsky, và tôi đã viết một bài b..."
